# Contact Tracing

## UTILS

In [8]:
import random
import string

def get_random_alphaNumeric_string(stringLength=8):
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))

len(get_random_alphaNumeric_string(32))

32

In [9]:
import hashlib
import hmac
from math import ceil

hash_len = 32
def hmac_sha256(key, data):
    return hmac.new(key, data, hashlib.sha256).digest()

def hkdf(length: int, ikm, salt: bytes = b"", info: bytes = b"") -> bytes:
    if len(salt) == 0:
        salt = bytes([0]*hash_len)
    prk = hmac_sha256(salt, ikm)
    t = b""
    okm = b""
    for i in range(ceil(length / hash_len)):
        t = hmac_sha256(prk, t + info + bytes([1+i]))
        okm += t
    return okm[:length]

# CLASSES OF IMPLIMENTATION

In [10]:
# MAKING KEYS
import datetime;



# REQUESTS TO SERVER
import requests
import json

import base64

#VIEWS
from termcolor import colored


# Class of the device of the user of app \d:D
class Device:
    def __init__(self, owner_name, device_id):
        self.device_id = device_id
        self.owner_name = owner_name
        self.__tab_daily_tracing_key = []
        
        self.__makeTracingKey() #generate __tracing_key
        self.__day_number = int(datetime.datetime.now().timestamp()/(60*60*24))
        self.__encodeDailyTracingKey() #generate __daily_tracing_key
        self.__tab_daily_tracing_key.append((self.__day_number, self.__daily_tracing_key))
        
        ##
        self.__tab_rolling_proxy_ids = []
        
        ##
        self.__get_infected = False
        
        ##
        self.__tab_infected_diag_keys = {}
        
        ###
        self.__tab_infected_diag_keys_tuples = []
        
        ###
        self.__infections_days = []
        
        
    def getOwnerName(self):
        return self.owner_name
    
    def getDeviceId(self):
        return self.device_id
    
    
    def __makeTracingKey(self, length=32):
            self.__tracing_key = get_random_alphaNumeric_string(length)
        
    def getTracingKey(self):
        return self.__tracing_key
    
    
    def __encodeDailyTracingKey(self):
        #print(self.__tracing_key)
        self.__daily_tracing_key = hkdf(length=16, ikm = bytes(self.__tracing_key, 'utf-8'), info = bytes(self.__day_number) )
        self.__tab_daily_tracing_key.append((self.__day_number, self.__daily_tracing_key))
        
        print(self.__tab_daily_tracing_key)
        
    def getDailyTraicingKey(self):
        if self.__day_number == int(datetime.datetime.now().timestamp()/(60*60*24)):
            return self.__daily_tracing_key
        
        self.__day_number = int(datetime.datetime.now().timestamp()/(60*60*24))
        
        #------update
        self.__encodeDailyTracingKey()
        
        return self.__daily_tracing_key
    
    
    def GetRollingProximityIdentifierKey(self):
        '''for now we gonna call this function to get proximity id each time'''
        
        #------update
        self.getDailyTraicingKey()
        
        
        seconds_since_start_of_day_number =  datetime.datetime.now().timestamp()% (60*60*24)
        
        self.__time_interval_number = int(seconds_since_start_of_day_number/(60*10)) 
        self.__time_interval_number_encoded = str(self.__time_interval_number).encode()
        print(self.__time_interval_number_encoded)
        
        self.__proximity_id_compete  = hmac.new(self.__daily_tracing_key, self.__time_interval_number_encoded , hashlib.sha256).digest()
        self.__proximity_id_16 = self.__proximity_id_compete[0:16]
        return self.__proximity_id_16
        
        
    def addContact(self, new_rolling_proxy_keys):
        if new_rolling_proxy_keys not in self.__tab_rolling_proxy_ids:
            self.__tab_rolling_proxy_ids.append(new_rolling_proxy_keys)
     
        
    def addContacts(self, new_rolling_proxy_keys):
        for proxy_key in new_rolling_proxy_keys:
            if proxy_key not in self.__tab_rolling_proxy_ids:
                self.__tab_rolling_proxy_ids.append(proxy_key)
        #extend
            
            
    def testedPositive(self, server_url = 'http://localhost:9000/positive' ):
        if not self.__get_infected:
            self.__get_infected = True
            
            
            
            self.__infetion_server = server_url

            
            diag_keys_list =[]
            
            for daily_tracy_key in self.__tab_daily_tracing_key:
                day_num = daily_tracy_key[0]
                diag_bytes = daily_tracy_key[1]
                
                diag_64 = base64.b64encode(diag_bytes).decode('ascii')
                
                diag_keys_list.append(
                    { 'day_num':day_num, 'diag_key': diag_64 } 
                )
            diagnosis_keys = {
                'data': diag_keys_list
            }
            
            response = requests.post(self.__infetion_server, data = json.dumps(diagnosis_keys)) 
            
            if response.status_code == 200:
                print('Success')
                print(response.text)
        
        
    def getInfectedDiagKeys(self,  server_url = 'http://localhost:9090/positive/all' ):

        self.__infetion_server = server_url
        response = requests.get(self.__infetion_server) 
        if response.status_code == 200:
            
            
            #it return some string keys so lets just chnage it for now at least
            self.__tab_infected_diag_keys = {}
            tab_infected_diag_keys_str = response.json()

            for num_day_str in sorted( tab_infected_diag_keys_str):
                if tab_infected_diag_keys_str[num_day_str] in self.__tab_daily_tracing_key and self.__get_infected==True:
                    print(self.getOwnerName(), ' Is Already infected self check from Cloud')
                    
                elif tab_infected_diag_keys_str[num_day_str] in self.__tab_daily_tracing_key and self.__get_infected==False:
                    print(self.getOwnerName(), ' Is not infected but his tracing still in Cloud as infected')
                    #later we will see if important to remove it from Cloud
                    
                elif tab_infected_diag_keys_str[num_day_str] not in self.__tab_daily_tracing_key and self.__get_infected==True:
                    self.testedPositive()
                else: 
                    self.__tab_infected_diag_keys[int(num_day_str)] = tab_infected_diag_keys_str[num_day_str]
            '''
            for num_day_str, infected_diag_key in  tab_infected_diag_keys_str.items():
                self.__tab_infected_diag_keys[int(num_day_str)] = infected_diag_key
            '''
            print('Success')
            #print(self.__tab_infected_diag_keys)

       
    def __generateRollingProximityIdentifierKey(self, infected_diag_key, time_interval_number):
        proximity_id_compete  = hmac.new(
            base64.b64decode(infected_diag_key) , 
            str(time_interval_number).encode()  , 
            hashlib.sha256).digest()
        
        proximity_id_16 = proximity_id_compete[0:16]
        return proximity_id_16
    
    def __generateAllPossibleRollingProximityIdentifierKey(self, nb = None):
        if nb is None:
            nb = len(self.__tab_infected_diag_keys)
        nb = min(len(self.__tab_infected_diag_keys), nb)    
        
        self.all_possible_rolling_proximity_key = []
        
        
        for num_day, infected_diag_key_list in list(self.__tab_infected_diag_keys.items())[-nb:]:
            self.__tab_infected_diag_keys_tuples.extend([(num_day, diag_key) for diag_key in infected_diag_key_list])
        
        self.all_possible_rolling_proximity_key = [(num_day, self.__generateRollingProximityIdentifierKey(diag_key, time_inter_num) )
                                                   for (num_day, diag_key) in self.__tab_infected_diag_keys_tuples for time_inter_num in range(144) ]
        
        #print(self.all_possible_rolling_proximity_key)
        
        
    def checkIfContactedInfectedDiagKeys(self):
        '''Here we give final situation of contact we give situation but Im not sure if we should give also the date '''
        
        #------update
        self.getInfectedDiagKeys()
        
        self.__generateAllPossibleRollingProximityIdentifierKey()
        
        
        for (num_day, gen_roll_prox_id_key ) in self.all_possible_rolling_proximity_key:
            for proxy_key in self.__tab_rolling_proxy_ids:
                if gen_roll_prox_id_key== proxy_key:
                    self.__infections_days.append(num_day)
                    
        if len(self.__infections_days)>0:
            for day in self.__infections_days :
                date =  date.fromtimestamp(day)
                print(colored('Alert ‼', 'red'), self.owner_name, "Has One contact Infected Persone at day : ", day)
                print(colored(self.owner_name+' Get a test if possible or self quarantine and and visit the hospital when facing Complications with infection', 'red'))
                #print(self.owner_name, "Has One contact infected at day : ", num_day)
        elif  self.__get_infected==True:
            print(colored('Double Alert 💥‼', 'red'), self.owner_name, " is Tested postive His infected")
            print(colored(self.owner_name+' Self qurantine and visit the hospital when facing Complications with infection', 'red'))
        else:
            
            print(colored('Ok ✔ ', 'green'), self.owner_name, " Has No contact with infected people")
            #print(self.owner_name, " Has No contact with infected people")
        
        #print(self.__tab_rolling_proxy_ids)
    
    



# TESTING DEVICES METHODES

In [11]:
d1 = Device("Mohamed Hassani", 111)

[(18645, b'\xc8\xa8x\xb0\xc8\x9f"\xd5\x86f{\x13l\xf7\xfd\xd6')]


In [12]:
#print(d1.getDailyTraicingKey())
print('Dailly tracing : ', d1.getDailyTraicingKey())
p_id = d1.GetRollingProximityIdentifierKey()
print('p_id ', p_id)
print(len(d1.GetRollingProximityIdentifierKey()))

Dailly tracing :  b'\xc8\xa8x\xb0\xc8\x9f"\xd5\x86f{\x13l\xf7\xfd\xd6'
b'141'
p_id  b'm\x06|\xd4\xa8\x1cT\xcc\r\x95,\x1d\x05lc\xf9'
b'141'
16


In [13]:

print(d1.getTracingKey())
print(d1.getOwnerName())
b = bytes(d1.getTracingKey(), 'utf-8')
print(b)
print(len(b))

h7dO6D7T59m5Jwv1FD1sKFLSDYh7k1ij
Mohamed Hassani
b'h7dO6D7T59m5Jwv1FD1sKFLSDYh7k1ij'
32


## Infections methodes

In [14]:
d1.getInfectedDiagKeys()

Success


In [15]:
d1.testedPositive()

Success
{
  "msg": "Successfully stored", 
  "status": 200
}



In [16]:
d1.checkIfContactedInfectedDiagKeys()

Success
Double Alert 💥‼ Mohamed Hassani  is Tested postive His infected
Mohamed Hassani Self qurantine and visit the hospital when facing Complications with infection


# Pulic meeting 

In [17]:
## HERE WE GONNA DEFINE SOME METING TO KNOW PEOPLE THAT MEET .... WE SEMULATE IN PRIMITIVE WAY THE BLE OF THE DEVICE 
### We need a table where exist all devices with an Id tha semulat BLE signal so ... 

class Meeting:
    
    def __init__(self):
        
        self.__tab_proxy_contact_tracing_people = [] # contain ids of devices in same pace ther is a BLE Link ...
        self.__tab_rolling_proximity_identifiers = [] # comtains rolling proximity identifiers generated by those devices every 10 min
    
    def getAllRollingProximityIdentifiers(self):
        return self.__tab_rolling_proximity_identifiers
    
    def addContact(self, device_id, devices_list): 
        '''remember that contact tracing is focusing on privacy here id device is just to semulat near distance contact between device that have Contact Tracing active'''
        if device_id not in self.__tab_proxy_contact_tracing_people:
            self.__tab_proxy_contact_tracing_people.append(device_id)
            
            for device in devices_list:
                if device.getDeviceId() == device_id:
                    device_rolling_proximity_identifier = device.GetRollingProximityIdentifierKey()
                    if device_rolling_proximity_identifier not in self.__tab_rolling_proximity_identifiers:
                        self.__tab_rolling_proximity_identifiers.append(device_rolling_proximity_identifier)
                    
            for device in devices_list:
                device.addContacts(self.__tab_rolling_proximity_identifiers)
                
            print('HI---------------------\n', self.__tab_rolling_proximity_identifiers)

In [18]:
# TEST MEETHING

devices = []

d2 = Device("Amine Reda", 222)
devices.append(d2)

d3 = Device("Meyiam Salhi", 333)
devices.append(d3)

meeting_1 = Meeting()

meeting_1.addContact(222, devices)
meeting_1.addContact(333, devices)


d2.checkIfContactedInfectedDiagKeys()
d3.checkIfContactedInfectedDiagKeys()
print(devices)

[(18645, b'j\xfam\x1c\xc3\xdf\x10m\xc8f06^z,P')]
[(18645, b'\xc2L\xac\x92\xa3\xb9 GN\xd0\xc9;\xbcc\xd6\xa3')]
b'141'
HI---------------------
 [b'\x7f\x82\xe7\xcc\xf6\xb2\xa8\x1bs\\\x02\n\xa3\x99\xfa\x07']
b'141'
HI---------------------
 [b'\x7f\x82\xe7\xcc\xf6\xb2\xa8\x1bs\\\x02\n\xa3\x99\xfa\x07', b'\xd6d\xde\xe3\t\xc78\x9f\x92J\xbc\xce8\x19\xb8a']
Success
Ok ✔  Amine Reda  Has No contact with infected people
Success
Ok ✔  Meyiam Salhi  Has No contact with infected people
[<__main__.Device object at 0x000001388FD57908>, <__main__.Device object at 0x000001388FD575C8>]


In [19]:
meeting_1.getAllRollingProximityIdentifiers()

[b'\x7f\x82\xe7\xcc\xf6\xb2\xa8\x1bs\\\x02\n\xa3\x99\xfa\x07',
 b'\xd6d\xde\xe3\t\xc78\x9f\x92J\xbc\xce8\x19\xb8a']

In [20]:
d2.testedPositive()

Success
{
  "msg": "Successfully stored", 
  "status": 200
}



In [21]:
d2.checkIfContactedInfectedDiagKeys()

Success
Double Alert 💥‼ Amine Reda  is Tested postive His infected
Amine Reda Self qurantine and visit the hospital when facing Complications with infection


In [22]:
d3.checkIfContactedInfectedDiagKeys()

Success
Ok ✔  Meyiam Salhi  Has No contact with infected people


In [23]:
d3.GetRollingProximityIdentifierKey()
meeting_1.addContact(333, devices)
meeting_1.getAllRollingProximityIdentifiers()

b'141'


[b'\x7f\x82\xe7\xcc\xf6\xb2\xa8\x1bs\\\x02\n\xa3\x99\xfa\x07',
 b'\xd6d\xde\xe3\t\xc78\x9f\x92J\xbc\xce8\x19\xb8a']